In [21]:
import pandas as pd
import time

wd = '/Users/ewenwang/Documents/practice_data/conversion_rate/'

file = ['round1_ijcai_18_train_20180301.txt', 'round1_ijcai_18_test_a_20180301.txt']

train, test = pd.read_csv(wd+file[0], sep=' '), pd.read_csv(wd+file[1], sep=' ')
data = pd.concat([train, test])

def timestamp_datetime(timestamp):
    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(timestamp))

data['realtime'] = pd.to_datetime(data['context_timestamp'].apply(timestamp_datetime))
data['day'] = data['realtime'].dt.day

In [25]:
feature_id = ['instance_id', 'user_id', 'item_id', 'context_id', 'shop_id', 'is_trade', 'day']

In [26]:
data_c = data[feature_id]

In [ ]:
def EngCVR(train, feature):
    """Calculate CVR features for categorical features."""
    
    feature_clk_df = pd.DataFrame(train.groupby([feature], as_index=False).size().reset_index())
    feature_clk_df.rename(columns={0:feature+'_count'}, inplace=True)

    feature_cv_df = pd.DataFrame(train[(train.is_trade==1)].groupby([feature], as_index=False).size().reset_index())
    feature_cv_df.rename(columns={0:feature+'_trade_count'}, inplace=True)

    feature_cvr_df = feature_clk_df.merge(feature_cv_df, on=feature, how='left')
    feature_cvr_df[feature+'_cvr'] = feature_cvr_df[feature+'_trade_count']/feature_cvr_df[feature+'_count']

    return feature_cvr_df

In [ ]:
feature_list = ['user_id', 'item_id', 'context_id', 'shop_id']

cvr_df = EngCVR(temp, feature_list)

In [ ]:
feature = 'user_id'

feature_clk_df = pd.DataFrame(temp.groupby([feature], as_index=False).size().reset_index())
feature_clk_df.rename(columns={0:feature+'_count'}, inplace=True)

feature_cv_df = pd.DataFrame(temp[(temp.is_trade==1)].groupby([feature], as_index=False).size().reset_index())
feature_cv_df.rename(columns={0:feature+'_trade_count'}, inplace=True)

feature_cvr_df = feature_clk_df.merge(feature_cv_df, on=feature, how='left')
feature_cvr_df[feature+'_cvr'] = feature_cvr_df[feature+'_trade_count']/feature_cvr_df[feature+'_count']

In [ ]:
feature_cvr_df.sort_values(by=['user_id_trade_count'], ascending=False).head()

In [ ]:
def EngCVR(line, feature):
    """Calculate CVR features for categorical features."""
    temp = train[(train.day == line.day-1) & (train[feature] == line[feature])]

    count = temp.shape[0]
    if count == 0:
        return None
    conversed= temp[(temp.is_trade==1)].shape[0]
    cvr = conversed/count

    return cvr

In [ ]:
train19 = train[(train.day==19)]
train19['user_cvr'] = train19.apply(lambda x: EngCVR(x, 'user_id'), axis=1)

In [ ]:
train19.sort_values(by='user_cvr', ascending=False)

In [2]:
def EngCVR(data, day, feature):
    """Calculate CVR features for categorical features."""
    
    data = data[(data.day == day)]
    feature_clk_df = pd.DataFrame(data.groupby([feature], as_index=False).size().reset_index())
    feature_clk_df.rename(columns={0:feature+'_count'}, inplace=True)

    feature_cv_df = pd.DataFrame(data[(data.is_trade==1)].groupby([feature], as_index=False).size().reset_index())
    feature_cv_df.rename(columns={0:feature+'_trade_count'}, inplace=True)

    feature_cvr_df = feature_clk_df.merge(feature_cv_df, on=feature, how='left')
    feature_cvr_df[feature+'_cvr'] = feature_cvr_df[feature+'_trade_count']/feature_cvr_df[feature+'_count']

    return feature_cvr_df

In [5]:
user_19 = EngCVR(train, 19, 'user_id')
user_19.sort_values(by='user_id_trade_count', ascending=False).head()

,user_id,user_id_count,user_id_trade_count,user_id_cvr
14851,3996770415362098116,7,3.0,0.428571
4897,1312449399120292157,4,2.0,0.500000
3986,1072283350770182831,8,2.0,0.250000
16393,4418327075663359810,3,2.0,0.666667
22979,6206783699089672711,6,2.0,0.333333


In [18]:
id_list = ['user_id', 'item_id', 'shop_id', 'context_id']
date_list = [18, 19, 20, 21, 22, 23, 24]

In [13]:
feature = 'user_id'

df = {}
for date in date_list:
    df['_'.join([feature, str(date)])] = EngCVR(train, date, feature) 

In [17]:
df['_'.join(['user_id', str(19)])].sort_values(by='user_id_trade_count', ascending=False).head(10)

,user_id,user_id_count,user_id_trade_count,user_id_cvr
14851,3996770415362098116,7,3.0,0.428571
4897,1312449399120292157,4,2.0,0.500000
3986,1072283350770182831,8,2.0,0.250000
16393,4418327075663359810,3,2.0,0.666667
22979,6206783699089672711,6,2.0,0.333333
21012,5680395797812000158,5,2.0,0.400000
15139,4081407113175208858,6,2.0,0.333333
20850,5640692426959486830,3,2.0,0.666667
26471,7126688615059572797,13,2.0,0.153846
16670,4488485750593727694,3,2.0,0.666667


In [46]:
id_list = ['user_id', 'item_id', 'shop_id', 'context_id']
date_list = [18, 19, 20, 21, 22, 23, 24]

df = {}

for feature in id_list:
    for date in date_list:
        df['_'.join([feature, str(date)])] = EngCVR(data_c, date, feature)
        

In [28]:
train[(train.day==19)] = train[(train.day==19)].merge(df['_'.join(['shop_id', str(18)])], on='shop_id', how='left')

In [33]:
train_19 = train[(train.day==19)]

In [32]:
df['_'.join(['shop_id', str(18)])]

,shop_id,shop_id_count,shop_id_trade_count,shop_id_cvr
0,1543559655939246,1,NaN,NaN
1,6515295840140210,7,1.0,0.142857
2,9230153951388399,11,NaN,NaN
3,10653466129289268,1,NaN,NaN
4,12057125006548226,12,NaN,NaN


In [47]:
date = 18
feature = 'shop_id'

df['_'.join([feature, str(date)])] = EngCVR(train, date, feature) 
df_temp = df['_'.join([feature, str(date)])]
train_19 = train_19.merge(df_temp, on=feature, how='left')

In [68]:
df_temp = EngCVR(data, date, feature) 
train_19 = train_19.merge(df_temp, on=feature, how='left')

In [69]:
train_19.head()

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,shop_id_cvr_y,shop_id_count_x,shop_id_trade_count_x,shop_id_cvr_x,shop_id_count_y,shop_id_trade_count_y,shop_id_cvr_y,context_id_count,context_id_trade_count,context_id_cvr
0,8020414902236616944,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,4,4,14,...,NaN,6.0,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN
1,4859094599505998036,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,4,4,14,...,NaN,6.0,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN
2,5558126838111646808,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,4,4,14,...,NaN,6.0,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN
3,847092414099368634,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,4,4,14,...,NaN,6.0,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN
4,8795328164245101288,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,4,4,14,...,NaN,6.0,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN


In [58]:
class CVR(object):
    """docstring for CVR"""
    def __init__(self, data):
        super(CVR, self).__init__()
        self.data = data

    def EngCVR(self, day, feature):
        """Calculate CVR features for categorical features."""
        
        temp = self.data[(self.data.day == day)]
        feature_clk_df = pd.DataFrame(temp.groupby([feature], as_index=False).size().reset_index())
        feature_clk_df.rename(columns={0:feature+'_count'}, inplace=True)

        feature_cv_df = pd.DataFrame(temp[(temp.is_trade==1)].groupby([feature], as_index=False).size().reset_index())
        feature_cv_df.rename(columns={0:feature+'_trade_count'}, inplace=True)

        feature_cvr_df = feature_clk_df.merge(feature_cv_df, on=feature, how='left')
        feature_cvr_df[feature+'_cvr'] = feature_cvr_df[feature+'_trade_count']/feature_cvr_df[feature+'_count']

        return feature_cvr_df

    def store(self):
        id_list = ['user_id', 'item_id', 'shop_id', 'context_id']
        date_list = [18, 19, 20, 21, 22, 23, 24]

        for date in date_list:
            for feature in id_list:
                df_add = self.EngCVR(day=date, feature=feature)
                self.data[(self.data.day==date+1)] = self.data[(self.data.day==date+1)].merge(df_add, on=feature, how='left')

        return self.data

In [59]:
cvr_data = CVR(data).store()

In [60]:
cvr_data.columns

Index(['context_id', 'context_page_id', 'context_timestamp', 'instance_id',
       'is_trade', 'item_brand_id', 'item_category_list', 'item_city_id',
       'item_collected_level', 'item_id', 'item_price_level',
       'item_property_list', 'item_pv_level', 'item_sales_level',
       'predict_category_property', 'shop_id', 'shop_review_num_level',
       'shop_review_positive_rate', 'shop_score_delivery',
       'shop_score_description', 'shop_score_service', 'shop_star_level',
       'user_age_level', 'user_gender_id', 'user_id', 'user_occupation_id',
       'user_star_level', 'realtime', 'day'],
      dtype='object')

In [44]:
def EngCVR(data, day, feature):
    """Calculate CVR features for categorical features."""
    
    data = data[(data.day == day)]
    feature_clk_df = pd.DataFrame(data.groupby([feature], as_index=False).size().reset_index())
    feature_clk_df.rename(columns={0:feature+'_count'}, inplace=True)

    feature_cv_df = pd.DataFrame(data[(data.is_trade==1)].groupby([feature], as_index=False).size().reset_index())
    feature_cv_df.rename(columns={0:feature+'_trade_count'}, inplace=True)

    feature_cvr_df = feature_clk_df.merge(feature_cv_df, on=feature, how='left')
    feature_cvr_df[feature+'_cvr'] = feature_cvr_df[feature+'_trade_count']/feature_cvr_df[feature+'_count']

    return feature_cvr_df

def store(data):
    id_list = ['user_id', 'item_id', 'shop_id', 'context_id']
    date_list = [18, 19, 20, 21, 22, 23, 24]
    new_data = pd.DataFrame()
    n = 1
    for date in date_list:
        date_df = data[(data.day==date+1)]
        for feature in id_list:
            df_add = EngCVR(data, day=date, feature=feature)
            date_df = date_df.merge(df_add, on=feature, how='left')
        if n==1:
            new_data = date_df
        else:
            new_data = pd.concat([new_data, date_df])
        n+=1
    
    return new_data

In [39]:
date = 18
id_list = ['user_id', 'item_id', 'shop_id', 'context_id']

date_df = data[(data.day==date+1)]

for feature in id_list:
    df_add = EngCVR(data, day=date, feature=feature)
    date_df = date_df.merge(df_add, on=feature, how='left')


In [43]:
date_df.columns

Index(['context_id', 'context_page_id', 'context_timestamp', 'instance_id',
       'is_trade', 'item_brand_id', 'item_category_list', 'item_city_id',
       'item_collected_level', 'item_id', 'item_price_level',
       'item_property_list', 'item_pv_level', 'item_sales_level',
       'predict_category_property', 'shop_id', 'shop_review_num_level',
       'shop_review_positive_rate', 'shop_score_delivery',
       'shop_score_description', 'shop_score_service', 'shop_star_level',
       'user_age_level', 'user_gender_id', 'user_id', 'user_occupation_id',
       'user_star_level', 'realtime', 'day', 'user_id_count',
       'user_id_trade_count', 'user_id_cvr', 'item_id_count',
       'item_id_trade_count', 'item_id_cvr', 'shop_id_count',
       'shop_id_trade_count', 'shop_id_cvr', 'context_id_count',
       'context_id_trade_count', 'context_id_cvr'],
      dtype='object')

In [52]:
cvr_data = store(data_c)

In [53]:
print(data.shape, cvr_data.shape)

(496509, 29) (418241, 19)


In [54]:
cvr_data.columns

Index(['instance_id', 'user_id', 'item_id', 'context_id', 'shop_id',
       'is_trade', 'day', 'user_id_count', 'user_id_trade_count',
       'user_id_cvr', 'item_id_count', 'item_id_trade_count', 'item_id_cvr',
       'shop_id_count', 'shop_id_trade_count', 'shop_id_cvr',
       'context_id_count', 'context_id_trade_count', 'context_id_cvr'],
      dtype='object')

In [55]:
cvr_data.head()

,instance_id,user_id,item_id,context_id,shop_id,is_trade,day,user_id_count,user_id_trade_count,user_id_cvr,item_id_count,item_id_trade_count,item_id_cvr,shop_id_count,shop_id_trade_count,shop_id_cvr,context_id_count,context_id_trade_count,context_id_cvr
0,8020414902236616944,4720121252002538475,3412720377098676069,4927988711139031691,6765930309048922341,0.0,19,NaN,NaN,NaN,6.0,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN
1,4859094599505998036,2807609768003682395,3412720377098676069,2008435332141948993,6765930309048922341,0.0,19,NaN,NaN,NaN,6.0,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN
2,5558126838111646808,5375117820682852541,3412720377098676069,5436638338323278037,6765930309048922341,0.0,19,NaN,NaN,NaN,6.0,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN
3,847092414099368634,5375117820682852541,3412720377098676069,138756866678114005,6765930309048922341,0.0,19,NaN,NaN,NaN,6.0,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN
4,8795328164245101288,2232016768046510105,3412720377098676069,7374456636392374908,6765930309048922341,0.0,19,NaN,NaN,NaN,6.0,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN


In [56]:
cvr_data.day.unique()

array([19, 20, 21, 22, 23, 24, 25])

In [61]:
drop_list = ['is_trade', 'user_id', 'item_id', 'context_id', 'shop_id', 'is_trade', 'day']
feature_cvr = [x for x in cvr_data.columns if x not in drop_list]

In [62]:
feature_cvr

['instance_id',
 'user_id_count',
 'user_id_trade_count',
 'user_id_cvr',
 'item_id_count',
 'item_id_trade_count',
 'item_id_cvr',
 'shop_id_count',
 'shop_id_trade_count',
 'shop_id_cvr',
 'context_id_count',
 'context_id_trade_count',
 'context_id_cvr']

In [63]:
cvr_data = cvr_data[feature_cvr]

In [67]:
cvr_data.sort_values(by='user_id_cvr', ascending=False).head()

,instance_id,user_id_count,user_id_trade_count,user_id_cvr,item_id_count,item_id_trade_count,item_id_cvr,shop_id_count,shop_id_trade_count,shop_id_cvr,context_id_count,context_id_trade_count,context_id_cvr
35464,3972957473546523928,1.0,1.0,1.0,34.0,NaN,NaN,213.0,NaN,NaN,NaN,NaN,NaN
44572,4080464836165785986,1.0,1.0,1.0,145.0,4.0,0.027586,157.0,5.0,0.031847,NaN,NaN,NaN
42073,2297192757045056212,1.0,1.0,1.0,113.0,NaN,NaN,282.0,1.0,0.003546,NaN,NaN,NaN
57931,7481436210746028681,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
58314,1550412857134241203,1.0,1.0,1.0,11.0,NaN,NaN,90.0,1.0,0.011111,NaN,NaN,NaN


In [68]:
cvr_data.to_csv(wd+'feature_cvr.txt', index=False, sep=' ')